In [8]:
class Txn:
    def __init__(self,  write_set=set(), read_set=set(),should_abort=False):
        self.read_set = read_set
        self.write_set = write_set
        self.should_abort = should_abort

In [9]:
# Sample data
# Lock table:
# X: [A, E]
# Y: [B]
# Z: [C, D, E]
# our adjacency list should look like this:
# A: [E]
# B: []
# C: [D, E]
# D: [E]

In [11]:
A = Txn({"X"})
B = Txn({"Y"})
C = Txn({"Z"})
D = Txn({"Z"})
E = Txn({"Z", "X"})

In [12]:
txns = [A, B, C, D, E]

In [ ]:
# 1) build the lock table
